In [6]:
import pandas as pd 
import numpy as numpy

#charger la list des C.V (data.csv)abs
data = pd.read_csv('clean_cv/data.csv')

df = pd.DataFrame(data)

# Afficherd les noms des colonnes
print(df.columns.tolist())



['Types de métiers recherchés', 'Expérience professionnelle', 'Compétences', 'Formation', 'Compétences clés', 'Langues', "Plus d'informations"]


In [7]:
df = df.drop(columns=["Plus d'informations", "Langues"])
print(df.columns.tolist())


['Types de métiers recherchés', 'Expérience professionnelle', 'Compétences', 'Formation', 'Compétences clés']


In [8]:
print(df.head())

                         Types de métiers recherchés  \
0               Informatique, nouvelles technologies   
1   Commercial, vente Gestion, comptabilité, fina...   
2               Informatique, nouvelles technologies   
3   Informatique, nouvelles technologies Métiers ...   
4               Informatique, nouvelles technologies   

                          Expérience professionnelle  \
0   Expérience dans les secteurs d´activité suiva...   
1   Expérience dans les secteurs d´activité suiva...   
2   Expérience dans les secteurs d´activité suiva...   
3   Expérience dans les secteurs d´activité suiva...   
4   Expérience dans les secteurs d´activité suiva...   

                                         Compétences  \
0   DÉVELOPPEMENT WEB, NOTAMMENT AVEC SPRING BOOT...   
1   Développement web Full Stack (WordPress, Next...   
2   vba excel dashboards python web html css js p...   
3   Langages informatiques: Python, Java, Dart Fr...   
4   ReactJS,NextJs,Vuejs ,Expressjs , Nodejs, 

In [9]:
# Vérifier les types de données et les valeurs manquantes
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Types de métiers recherchés  985 non-null    object
 1   Expérience professionnelle   985 non-null    object
 2   Compétences                  983 non-null    object
 3   Formation                    985 non-null    object
 4   Compétences clés             812 non-null    object
dtypes: object(5)
memory usage: 38.6+ KB
None


In [10]:
# Statistiques descriptives pour les colonnes numériques
print(df.describe())

                  Types de métiers recherchés  \
count                                     985   
unique                                    226   
top      Informatique, nouvelles technologies   
freq                                      541   

                               Expérience professionnelle Compétences  \
count                                                 985         983   
unique                                                801         982   
top      Expérience dans les secteurs d´activité suiva...           .   
freq                                                  116           2   

                                                Formation  \
count                                                 985   
unique                                                985   
top      Niveau d'études : Bac+5 et plus GENIE LOGICIE...   
freq                                                    1   

                  Compétences clés  
count                          812  
unique  

In [11]:
#les valeurs manquantes
print(df.isnull().sum())

Types de métiers recherchés      0
Expérience professionnelle       0
Compétences                      2
Formation                        0
Compétences clés               173
dtype: int64


In [12]:
#Remplacer les valeurs manquantes par "Non spécifié" 
df["Compétences"] = df["Compétences"].fillna("Non spécifié")
df["Compétences clés"] = df["Compétences clés"].fillna("Non spécifié")

print(df.isnull().sum())

Types de métiers recherchés    0
Expérience professionnelle     0
Compétences                    0
Formation                      0
Compétences clés               0
dtype: int64


In [13]:
import re

# Nettoyer les colonnes  (retirer les emojis, ponctuation et symboles inutiles)
df['Compétences'] = df['Compétences'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Compétences clés'] = df['Compétences clés'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Expérience professionnelle'] = df['Expérience professionnelle'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Formation'] = df['Formation'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

print(df.describe())

                  Types de métiers recherchés  \
count                                     985   
unique                                    226   
top      Informatique, nouvelles technologies   
freq                                      541   

                               Expérience professionnelle Compétences  \
count                                                 985         985   
unique                                                801         982   
top      Expérience dans les secteurs dactivité suivan...               
freq                                                  116           3   

                                                Formation Compétences clés  
count                                                 985              985  
unique                                                985              733  
top      Niveau détudes  Bac5 et plus GENIE LOGICIELMI...     Non spécifié  
freq                                                    1              173  


In [14]:
print(df.isnull().sum())

Types de métiers recherchés    0
Expérience professionnelle     0
Compétences                    0
Formation                      0
Compétences clés               0
dtype: int64


In [15]:
resume = df
resume["cv_resume"] = resume["Expérience professionnelle"].fillna('') + " " + resume["Compétences"].fillna('') + " " + resume["Formation"].fillna('') + " " + resume["Compétences clés"].fillna('')
print(resume.head())


                         Types de métiers recherchés  \
0               Informatique, nouvelles technologies   
1   Commercial, vente Gestion, comptabilité, fina...   
2               Informatique, nouvelles technologies   
3   Informatique, nouvelles technologies Métiers ...   
4               Informatique, nouvelles technologies   

                          Expérience professionnelle  \
0   Expérience dans les secteurs dactivité suivan...   
1   Expérience dans les secteurs dactivité suivan...   
2   Expérience dans les secteurs dactivité suivan...   
3   Expérience dans les secteurs dactivité suivan...   
4   Expérience dans les secteurs dactivité suivan...   

                                         Compétences  \
0   DÉVELOPPEMENT WEB NOTAMMENT AVEC SPRING BOOT ...   
1   Développement web Full Stack WordPress Nextjs...   
2   vba excel dashboards python web html css js p...   
3   Langages informatiques Python Java Dart Frame...   
4   ReactJSNextJsVuejs Expressjs  Nodejs Redux

In [16]:
#Applique S-BERT sur le C.V global
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Chargement du modèle pré-entraîné (par exemple, paraphrase-MiniLM-L6-v2)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [17]:
offre_emploi = "Recherche développeur full-stack avec compétences en Spring Boot et React."

# Embedding pour l'offre d'emploi
offre_embedding = model.encode(offre_emploi).reshape(1,-1) 

# Embedding pour les cvs
cv_embeddings = model.encode(resume["cv_resume"].tolist()) 

# Calcul de la similarité cosinus entre l'offre d'emploi et chaque CV
similarities = cosine_similarity(offre_embedding, cv_embeddings)[0]

top_5_indices = np.argsort(similarities)[-5:][::-1]

# Affichage des résultats de similarité
for i in top_5_indices:
    print(f"Similarité avec CV {i+1}: {similarities[i]:.4f}")

Similarité avec CV 1: 0.6296
Similarité avec CV 733: 0.5914
Similarité avec CV 547: 0.5459
Similarité avec CV 333: 0.5418
Similarité avec CV 391: 0.5370


In [18]:
# applique S-BERT sur chauqe section individuel 

# Embeddings et similarités par section
cv_embeddings_expr = model.encode(resume["Expérience professionnelle"].tolist())
cv_embeddings_cmpet = model.encode(resume["Compétences"].tolist())
cv_embeddings_edu = model.encode(resume["Formation"].tolist())
cv_embeddings_cmpt_cle = model.encode(resume["Compétences clés"].tolist())

# calcul similarité de chaque section
similarities_expr = cosine_similarity(offre_embedding, cv_embeddings_expr)[0]
similarities_cmpet = cosine_similarity(offre_embedding, cv_embeddings_cmpet)[0]
similarities_edu = cosine_similarity(offre_embedding, cv_embeddings_edu)[0]
similarities_cmpet_cle = cosine_similarity(offre_embedding, cv_embeddings_cmpt_cle)[0]

top_5_expr = np.argsort(similarities_expr)[-5:][::-1]
top_5_cmpet = np.argsort(similarities_cmpet)[-5:][::-1]
top_5_edu = np.argsort(similarities_edu)[-5:][::-1]
top_5_cmpet_cle = np.argsort(similarities_cmpet_cle)[-5:][::-1]

# calcul simularite global 
# Poids attribués à chaque section (La relation de la moyenne pondérée )
weights = {
    "Expérience professionnelle": 0.4,
    "Compétences": 0.35,
    "Formation": 0.05,
    "Compétences clés": 0.2
}

# Calcul de la similarité globale en appliquant la moyenne pondérée
similarities_global = (
    weights["Expérience professionnelle"] * similarities_expr +
    weights["Compétences"] * similarities_cmpet +
    weights["Formation"] * similarities_edu +
    weights["Compétences clés"] * similarities_cmpet_cle
)

# Extraction des 5 CVs les plus similaires globalement
top_5_global = np.argsort(similarities_global)[-5:][::-1]



In [19]:
#afficher les 5 premier grand similarité de chaque section

# Affichage des résultats de similarité pour la section Expérience professionnelle
print("--------------------------------------------------------------")
print("\n")

for i in top_5_expr:
    print(f"Expérience professionnelle | Similarité avec CV {i+1}: {similarities_expr[i]:.4f}")
    
print("--------------------------------------------------------------")
print("\n")

# Affichage des résultats de similarité pour la section Compétences
for i in top_5_cmpet:
    print(f"Compétences | Similarité avec CV {i+1}: {similarities_cmpet[i]:.4f}")
    
print("--------------------------------------------------------------")
print("\n")

# Affichage des résultats de similarité pour la section Compétences
for i in top_5_edu:
    print(f"Formation | Similarité avec CV {i+1}: {similarities_edu[i]:.4f}")
    
print("--------------------------------------------------------------")
print("\n")

# Affichage des résultats de similarité pour la section Compétences
for i in top_5_cmpet_cle:
    print(f"Compétences clés | Similarité avec CV {i+1}: {similarities_cmpet_cle[i]:.4f}")
    
print("--------------------------------------------------------------")
print("\n")

for i in top_5_global:
    print(f"CV {i+1} | Similarité Globale : {similarities_global[i]:.4f}")





--------------------------------------------------------------


Expérience professionnelle | Similarité avec CV 104: 0.5492
Expérience professionnelle | Similarité avec CV 835: 0.5428
Expérience professionnelle | Similarité avec CV 745: 0.5215
Expérience professionnelle | Similarité avec CV 46: 0.5211
Expérience professionnelle | Similarité avec CV 479: 0.5145
--------------------------------------------------------------


Compétences | Similarité avec CV 445: 0.7246
Compétences | Similarité avec CV 1: 0.6574
Compétences | Similarité avec CV 853: 0.5657
Compétences | Similarité avec CV 136: 0.5606
Compétences | Similarité avec CV 220: 0.5419
--------------------------------------------------------------


Formation | Similarité avec CV 417: 0.5160
Formation | Similarité avec CV 152: 0.4555
Formation | Similarité avec CV 697: 0.4484
Formation | Similarité avec CV 438: 0.4385
Formation | Similarité avec CV 403: 0.4371
--------------------------------------------------------------


Com

In [20]:
#applique de tf-idf globalemnent
# import required module
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialiser le vectoriseur TF-IDF
vectorizer = TfidfVectorizer()

offre_emploi = "Recherche développeur full-stack avec compétences en Spring Boot et React."

# Vectoriser les CVs et l'offre d'emploi
tfidf_matrix = vectorizer.fit_transform(resume['cv_resume'])
offre_emploi_vector = vectorizer.transform([offre_emploi])

# Calculer la similarité cosinus entre l'offre d'emploi et chaque CV
similarities_tf_idf = cosine_similarity(offre_emploi_vector, tfidf_matrix)[0]

top_5_indices_tf_idf = np.argsort(similarities_tf_idf)[-5:][::-1]

# Affichage des résultats de similarité
for i in top_5_indices_tf_idf:
    print(f"Similarité avec CV {i+1}: {similarities_tf_idf[i]:.4f}")

Similarité avec CV 948: 0.4060
Similarité avec CV 197: 0.3786
Similarité avec CV 435: 0.3747
Similarité avec CV 834: 0.3558
Similarité avec CV 46: 0.3250


In [21]:
#applique de tf-idf sur chaque section
tfidf_matrix_exp = vectorizer.transform(resume['Expérience professionnelle'])
tfidf_matrix_cmpt = vectorizer.transform(resume['Compétences'])
tfidf_matrix_edu = vectorizer.transform(resume['Formation'])
tfidf_matrix_cmpt_cle = vectorizer.transform(resume['Compétences clés'])

similarities_tf_idf_exp = cosine_similarity(offre_emploi_vector, tfidf_matrix_exp)[0]
similarities_tf_idf_cmpt = cosine_similarity(offre_emploi_vector, tfidf_matrix_cmpt)[0]
similarities_tf_idf_edu = cosine_similarity(offre_emploi_vector, tfidf_matrix_edu)[0]
similarities_tf_idf_cmpt_cle = cosine_similarity(offre_emploi_vector, tfidf_matrix_cmpt_cle)[0]


top_5_indices_tf_idf_exp = np.argsort(similarities_tf_idf_exp)[-5:][::-1]
top_5_indices_tf_idf_cmpt = np.argsort(similarities_tf_idf_cmpt)[-5:][::-1]
top_5_indices_tf_idf_edu = np.argsort(similarities_tf_idf_edu)[-5:][::-1]
top_5_indices_tf_idf_cmpt_cle = np.argsort(similarities_tf_idf_cmpt_cle)[-5:][::-1]

# Calcul de la similarité globale en appliquant la moyenne pondérée
similarities_global_tf_idf = (
    weights["Expérience professionnelle"] * similarities_tf_idf_exp +
    weights["Compétences"] * similarities_tf_idf_cmpt +
    weights["Formation"] * similarities_tf_idf_edu +
    weights["Compétences clés"] * similarities_tf_idf_cmpt_cle
)

# Extraction des 5 CVs les plus similaires globalement
top_5_global_tf_idf = np.argsort(similarities_global_tf_idf)[-5:][::-1]




In [20]:
# Affichage des résultats de similarité
print("--------------------------------------------------------------")
print("\n")

for i in top_5_indices_tf_idf_exp:
    print(f"Expérience professionnelle | Similarité avec CV {i+1}: {similarities_tf_idf_exp[i]:.4f}")

print("--------------------------------------------------------------")
print("\n")

for i in top_5_indices_tf_idf_cmpt:
    print(f"Compétence | Similarité avec CV {i+1}: {similarities_tf_idf_cmpt[i]:.4f}")

print("--------------------------------------------------------------")
print("\n")

for i in top_5_indices_tf_idf_edu:
    print(f"Formation | Similarité avec CV {i+1}: {similarities_tf_idf_edu[i]:.4f}")

print("--------------------------------------------------------------")
print("\n")

for i in top_5_indices_tf_idf_cmpt_cle:
    print(f"Competence cle | Similarité avec CV {i+1}: {similarities_tf_idf_cmpt_cle[i]:.4f}")

print("--------------------------------------------------------------")
print("\n")

for i in top_5_global_tf_idf:
    print(f"CV {i+1} | Similarité Globale : {similarities_global_tf_idf[i]:.4f}")

--------------------------------------------------------------


Expérience professionnelle | Similarité avec CV 197: 0.3816
Expérience professionnelle | Similarité avec CV 851: 0.3553
Expérience professionnelle | Similarité avec CV 46: 0.3387
Expérience professionnelle | Similarité avec CV 104: 0.3298
Expérience professionnelle | Similarité avec CV 835: 0.3027
--------------------------------------------------------------


Compétence | Similarité avec CV 923: 0.5701
Compétence | Similarité avec CV 138: 0.5467
Compétence | Similarité avec CV 333: 0.5052
Compétence | Similarité avec CV 136: 0.4728
Compétence | Similarité avec CV 762: 0.4335
--------------------------------------------------------------


Formation | Similarité avec CV 124: 0.3707
Formation | Similarité avec CV 834: 0.3577
Formation | Similarité avec CV 435: 0.3548
Formation | Similarité avec CV 417: 0.3349
Formation | Similarité avec CV 917: 0.3014
--------------------------------------------------------------


Compet